<a href="https://colab.research.google.com/github/Devicharith/Deep-Fake-Bot/blob/master/Deep_Fake_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Clone the Repositorie



In [1]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 259, done.
remote: Total 259 (delta 0), reused 0 (delta 0), pack-reused 259
Receiving objects: 100% (259/259), 72.13 MiB | 32.81 MiB/s, done.
Resolving deltas: 100% (131/131), done.


Add folder https://drive.google.com/drive/folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw?usp=sharing to your google drive.

Mount your Google drive folder on Colab

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Load driving video and source image

In [5]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

#Add custom photo and driving video from your Google Drive here
source_url= '/content/gdrive/MyDrive/first-order-motion-model/rdj1.png'
driving_url = '/content/gdrive/MyDrive/first-order-motion-model/obama.mp4'

source_image = imageio.imread(source_url)
driving_video = imageio.mimread(driving_url, memtest=False)

#Resize image and video to 256x256
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

HTML(display(source_image, driving_video).to_html5_video())

Create a model and load checkpoints

In [6]:
cd first-order-model

/content/first-order-model


In [7]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', checkpoint_path='/content/gdrive/MyDrive/first-order-motion-model/vox-adv-cpk.pth.tar')

Perform image animation

In [8]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

#get fps from driving video
reader = imageio.get_reader(driving_url)
FPS = reader.get_meta_data()['fps']

#save resulting video
imageio.mimsave('/content/gdrive/MyDrive/first-order-motion-model/ani-output.mp4', [img_as_ubyte(frame) for frame in predictions], fps=FPS)
#video can be downloaded from /content folder

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 663/663 [00:25<00:00, 25.98it/s]


Audio to the animation

In [9]:
import moviepy.editor as mpe

video = mpe.VideoFileClip('/content/gdrive/MyDrive/first-order-motion-model/ani-output.mp4')

#extract audio from original driving video
audio = mpe.AudioFileClip(driving_url)

#set audio into your output video
Animation = video.set_audio(audio)
Animation.write_videofile("/content/gdrive/MyDrive/first-order-motion-model/afinal-out.mp4",fps=FPS)

# merge Original and Animation
original = mpe.VideoFileClip(driving_url).resize((250,250))
merge  = mpe.clips_array([[original,Animation]])
merge.write_videofile("/content/gdrive/MyDrive/first-order-motion-model/afinal-out-compare.mp4",fps=FPS)

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3219456/45929032 bytes (7.0%)7151616/45929032 bytes (15.6%)11173888/45929032 bytes (24.3%)15155200/45929032 bytes (33.0%)19013632/45929032 bytes (41.4%)22970368/45929032 bytes (50.0%)26935296/45929032 bytes (58.6%)30990336/45929032 bytes (67.5%)35037184/45929032 bytes (76.3%)39215104/45929032 bytes (85.4%)43040768/45929032 bytes (93.7%)45929032/45929032 bytes (100.0%)
  Done
File saved as /root

100%|██████████| 488/488 [00:00<00:00, 905.36it/s]

[MoviePy] Done.
[MoviePy] Writing video /content/gdrive/MyDrive/first-order-motion-model/afinal-out.mp4



100%|██████████| 663/663 [00:02<00:00, 256.36it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/gdrive/MyDrive/first-order-motion-model/afinal-out.mp4 

[MoviePy] >>>> Building video /content/gdrive/MyDrive/first-order-motion-model/afinal-out-compare.mp4
[MoviePy] Writing audio in afinal-out-compareTEMP_MPY_wvf_snd.mp3


100%|██████████| 488/488 [00:00<00:00, 590.81it/s]

[MoviePy] Done.
[MoviePy] Writing video /content/gdrive/MyDrive/first-order-motion-model/afinal-out-compare.mp4



100%|██████████| 664/664 [00:18<00:00, 36.87it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/gdrive/MyDrive/first-order-motion-model/afinal-out-compare.mp4 



Original and Deep Fake

In [10]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open( '/content/gdrive/MyDrive/first-order-motion-model/afinal-out-compare.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=300 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

Deep Fake

In [11]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/gdrive/MyDrive/first-order-motion-model/afinal-out.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=200 height = 150 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)